# Market Basket Anlysis Dengan Algoritma Apriori dan FP-Growth pada Data Online Retail

Nama Kelompok:
- Ni Luh Putu Yonita Putri Utami  (2105551066)
- Ni Putu Triska Widiantari (2105551087)

## Feature Engineering

### 1. Overview
- Market Basket Anlysis adalah salah satu teknik untuk mengetahui hubungan antar item
- Market Basket Analysis bekerja dengan mencari kombinasi item yang sering terjadi bersamaan dalam transaksi
- Market Basket Analysis menggunakan metode Association Rules, metode ini digunakan untuk menganalisis market basket atau dat atransaksi, dan dimaksukan untuk mengidentifikasikan rules yang kuat dalam data transaksi menggunakan ketertarikan.

### 2. Load Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import datetime as dt

# Parameter Konfigurasi Runtime untuk Matplotlib
plt.rcParams['font.family'] = 'Verdana'
plt.style.use('ggplot')

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Read data
df = pd.read_csv('OnlineRetail.csv', encoding = 'unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


#### 2.1 About the data

1. InvoiceNo: 6 digit bilangan yang ditetapkan secara unik untuk setiap transaksi. Jika kode ini dimulai dengan huruf 'c', berarti pembatalan.
2. StockCode: 5 digit bilangan yang ditetapkan secara unik untuk setiap produk berbeda.
3. Description: nama produk atau item
4. Quantity: Jumlah setiap produk atau item per transaksi
5. InvoiceDate: hari dan waktu terjadinya setiap transaksi
6. UnitPrice: harga produk per unit
7. CustomerID: 5 digit bilangan yang unik untuk setiap konsumen
8. Country: nama dari negara (tempat tinggal konsumen)

In [4]:
print('Jumlah baris dataset: ', df.shape[0])
print('Jumlah kolom dataset: ', df.shape[1])

Jumlah baris dataset:  541909
Jumlah kolom dataset:  8


##### 2.1.1 Missing values

In [5]:
def missing_values(df):
    total_null = df.isna().sum()
    missing_data = pd.concat([total_null], axis = 1, keys = ['Total missing values'])
    return missing_data

missing_values(df)

,Total missing values
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


### 3. Feature Engineering

#### 3.1 Data cleaning

In [6]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

##### 3.1.1 CustomerID

In [7]:
# Mengisi nilai NA dengan 0
df['CustomerID'] = df['CustomerID'].fillna(0)

# Mengubah tipe data dari float ke int
df['CustomerID'] = df['CustomerID'].astype(int)

# Mengubah tipe data dari int ke str
df['CustomerID'] = df['CustomerID'].astype(str)

# Mengganti nilai '0' menjadi 'Guest Customer'
df['CustomerID'] = df['CustomerID'].replace('0', 'Guest Customer')


In [8]:
df['CustomerID'].value_counts()

CustomerID
Guest Customer    135080
17841               7983
14911               5903
14096               5128
12748               4642
                   ...  
13270                  1
17763                  1
17291                  1
15668                  1
15562                  1
Name: count, Length: 4373, dtype: int64

##### 3.1.2 InvoiceID

In [9]:
# Tambah kolom 'Date' dengan tanggal dari InvoiceDate
df['Date'] = df['InvoiceDate'].dt.date

# Tambah kolom 'Time' dengan waktu dari InvoiceDate
df['Time'] = df['InvoiceDate'].dt.time

# Tambah kolom 'Hour' dengan jam (dalam format 24-jam) dari InvoiceDate
df["Hour"] = df['InvoiceDate'].dt.strftime('%H')

# Tambah kolom 'Time of Day' berdasarkan jam pada InvoiceDate (Pagi, Siang, Malam)
df['Time of Day'] = pd.cut(df['InvoiceDate'].dt.hour, [0,6,12,18,23], labels = ['Night','Morning','Afternoon','Evening'], include_lowest=True)

# Tambah kolom 'Month' dengan bulan dari InvoiceDate (nama bulan)
df['Month'] = df['InvoiceDate'].dt.month
df['Month'] = df['Month'].apply(lambda x: calendar.month_name[int(x)])

# Tambah kolom 'Year' dengan tahun dari InvoiceDate
df['Year'] = df['InvoiceDate'].dt.year

# Tambah kolom 'Week of the Year' dengan minggu ke berapa dalam tahun dari InvoiceDate
df['Week of the Year'] = df['InvoiceDate'].dt.isocalendar().week

# Tambah kolom 'Day of Week' dengan hari dalam seminggu dari InvoiceDate
df['Day of Week'] = df['InvoiceDate'].dt.day_name()

# Drop NA (Missing Values) dari kolom 'InvoiceNo'
df['InvoiceNo'] = df['InvoiceNo'].dropna()


In [10]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Time,Hour,Time of Day,Month,Year,Week of the Year,Day of Week
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010-12-01,08:26:00,08,Morning,December,2010,48,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,08:26:00,08,Morning,December,2010,48,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,2010-12-01,08:26:00,08,Morning,December,2010,48,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,08:26:00,08,Morning,December,2010,48,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010-12-01,08:26:00,08,Morning,December,2010,48,Wednesday


In [11]:
# Jangka waktu pada data
print('Tanggal terlama dalam dataset: ', min(df['Date']))
print('Tanggal terbaru dalam dataset: ', max(df['Date']))

Tanggal terlama dalam dataset:  2010-12-01
Tanggal terbaru dalam dataset:  2011-12-09


In [12]:
# Hapus baris dari tahun 2010, Jadi kita hanya menggunakan data tahun 2011 saja
index_names = df[df['Year'] == 2010].index
df.drop(index_names, inplace = True)
df.drop('Year', axis=1, inplace=True)

In [13]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Time,Hour,Time of Day,Month,Week of the Year,Day of Week
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday
42482,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday
42483,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday
42484,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday


##### 3.1.3 Sales revenue

In [14]:
# Jumlah Total Uang yang diperoleh dari penjualan produk
df['Sales Revenue'] = df['UnitPrice'] * df['Quantity']

# Periksa apakah Sales Revenue adalah negatif
index_names = df[df['Sales Revenue'] <= 0].index
df.drop(index_names, inplace = True)
# Jika terdapat nilai negatif maka akan kita drop
df.drop_duplicates(inplace = True)

##### 3.1.4 Country

In [15]:
print(df['Country'].unique())

['United Kingdom' 'Sweden' 'France' 'Australia' 'Germany' 'Spain'
 'Portugal' 'EIRE' 'Israel' 'Cyprus' 'Belgium' 'Italy' 'Netherlands'
 'Poland' 'Switzerland' 'Finland' 'Greece' 'Hong Kong' 'Singapore'
 'Iceland' 'Channel Islands' 'Lebanon' 'Austria' 'Norway' 'Japan'
 'United Arab Emirates' 'Denmark' 'Saudi Arabia' 'Czech Republic' 'Canada'
 'Unspecified' 'Brazil' 'USA' 'European Community' 'Bahrain' 'Malta' 'RSA']


In [16]:
# Drop baris dengan country == 'Unspecified'
df.drop(df[df['Country'] == 'Unspecified'].index, inplace=True)

##### 3.1.5 Quantity

In [17]:
# Quantity harus lebih dari 0
drop_quantity = df[df['Quantity'] <= 0].index
df.drop(drop_quantity, inplace = True)

##### 3.1.6 Description

In [18]:
print(df['Description'].unique())

['JUMBO BAG PINK POLKADOT' 'BLUE POLKADOT WRAP' 'RED RETROSPOT WRAP ' ...
 'LETTER "U" BLING KEY RING' 'CREAM HANGING HEART T-LIGHT HOLDER'
 'PAPER CRAFT , LITTLE BIRDIE']


In [19]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Time,Hour,Time of Day,Month,Week of the Year,Day of Week,Sales Revenue
42481,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,19.5
42482,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,10.5
42483,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,10.5
42484,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,10.5
42485,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313,United Kingdom,2011-01-04,10:00:00,10,Morning,January,1,Tuesday,12.5


In [20]:
# Drop NaN
df.dropna(subset=['Description'], inplace=True)


Menghapus deskripsi yang terdiri dari 8 kata atau kurang sering dilakukan untuk membersihkan data dari entri yang kurang informatif atau tidak relevan. 

In [21]:
# Hapus baris dengan deskripsi <= 8 kata
df.drop(df[df['Description'].str.len() <= 8].index, inplace = True)
df['Description'] = df['Description'].str.strip()
df.reset_index(drop=True, inplace=True)

#### Data setelah cleaning

In [22]:
print('Jumlah baris dataset setelah cleaning data: ', df.shape[0])
print('Jumlah kolom dataset setelah cleaning data: ', df.shape[1])

Jumlah baris dataset setelah cleaning data:  481966
Jumlah kolom dataset setelah cleaning data:  16


In [23]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Date', 'Time', 'Hour',
       'Time of Day', 'Month', 'Week of the Year', 'Day of Week',
       'Sales Revenue'],
      dtype='object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481966 entries, 0 to 481965
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         481966 non-null  object        
 1   StockCode         481966 non-null  object        
 2   Description       481966 non-null  object        
 3   Quantity          481966 non-null  int64         
 4   InvoiceDate       481966 non-null  datetime64[ns]
 5   UnitPrice         481966 non-null  float64       
 6   CustomerID        481966 non-null  object        
 7   Country           481966 non-null  object        
 8   Date              481966 non-null  object        
 9   Time              481966 non-null  object        
 10  Hour              481966 non-null  object        
 11  Time of Day       481966 non-null  category      
 12  Month             481966 non-null  object        
 13  Week of the Year  481966 non-null  UInt32        
 14  Day 

In [25]:
# Save cleaned daatset
df.to_csv('CleanOnlineRetail.csv')